# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

Recommender systems are software tools and techniques providing suggestions for
items to be of use to a user. The suggestions provided by a recommender system are
aimed at supporting their users in various decision-making processes, such as what
items to buy, what music to listen, or what news to read. Recommender systems
are valuable means for online users to cope with information overload and help
them making better choices. They are now one of the most powerful and popular
information discovery tools on the web. Several techniques for recommendation
generation have been proposed, and during the last decade, many of them have also
been successfully deployed in commercial environments.
Development of recommender systems is a multi-disciplinary effort which
involves experts from various fields such as artificial intelligence, human computer
interaction, data mining, statistics, decision support systems, marketing, and consumer behavior.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [13]:
grouped = data.groupby(["CustomerID", "ProductName"]).sum()
grouped = grouped.reset_index()
grouped

,CustomerID,ProductName,SalesID,ProductID,Quantity
0,33,Apricots - Dried,172592,324,1
1,33,Assorted Desserts,5363705,219,1
2,33,Bandage - Flexible Neon,6299165,273,1
3,33,"Bar Mix - Pina Colada, 355 Ml",3147536,281,1
4,33,"Beans - Kidney, Canned",6507033,300,1
...,...,...,...,...,...
63623,98200,Vol Au Vents,5160346,272,50
63624,98200,Wasabi Powder,4438427,80,25
63625,98200,Wine - Fume Blanc Fetzer,6226060,76,25
63626,98200,Wine - Hardys Bankside Shiraz,266676,360,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [14]:
pivot_grouped = grouped.pivot_table(index="ProductName", columns= "CustomerID", values ="Quantity", fill_value = 0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [15]:
squareform(pdist(pivot_grouped.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [16]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_grouped.T, 'euclidean'))), 
                         index=pivot_grouped.columns, columns=pivot_grouped.columns)

customer_33 = distances.loc[[33]].transpose()
customer_33 = customer_33.sort_values(by = 33, ascending=False)
customer_33 = customer_33.head(6)
customer_33

CustomerID,33
CustomerID,
33,1.000000
3317,0.087047
3535,0.087047
264,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [21]:
similar = grouped.query("CustomerID == 3317 or CustomerID == 3535  or CustomerID == 264 or CustomerID == 2503 or CustomerID == 3305")
similar

,CustomerID,ProductName,SalesID,ProductID,Quantity
131,264,Apricots - Halves,3730301,278,1
132,264,Apricots Fresh,5035449,158,1
133,264,Bacardi Breezer - Tropical,756802,49,1
134,264,Bagel - Plain,6728381,262,1
135,264,Banana - Leaves,6298459,271,1
...,...,...,...,...,...
3003,3535,Tray - 16in Rnd Blk,6122047,166,1
3004,3535,Wanton Wrap,2643670,253,1
3005,3535,Wine - Blue Nun Qualitatswein,3314104,269,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",6563480,192,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [23]:
a = similar.groupby(["ProductName", "ProductID"]).agg({"Quantity": "sum"})
a = a.Quantity.sort_values(ascending=False)
a = pd.DataFrame(a)
a = a.reset_index()
a

,ProductName,ProductID,Quantity
0,Towels - Paper / Kraft,319,3
1,Soup - Campbells Bean Medley,382,3
2,Wine - Blue Nun Qualitatswein,269,3
3,Cup - Translucent 7 Oz Clear,188,2
4,Pecan Raisin - Tarts,48,2
...,...,...,...
216,"Pepsi - Diet, 355 Ml",407,1
217,"Pepper - White, Ground",306,1
218,"Pepper - Paprika, Hungarian",60,1
219,"Pepper - Black, Whole",187,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [24]:
items_of_33 = list(grouped.loc[33].ProductID)

TypeError: 'numpy.int64' object is not iterable

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.